# Generate models for the scheduling experiments
In this notebook we generate the model parameters for the scheduling experiments

In [ ]:
import matplotlib.pyplot as plt
from IPython import display
from envs.mcom import SingleBSMComEnv
from envs.user_movement import PositionGenerator, RandomMovement
import numpy as np
from functools import partial
import numpy as np
import cvxpy as cp
from algorithms.MAB import MAB
from algorithms.reward_model import CustomRewardModel
from algorithms.fairness_model import UniformFairnessModel
from algorithms.fairness_model import VectorFairnessModel
from algorithms.bai.fbai import FairBAI
from algorithms.bai.tas import TaS
from algorithms.bai.uniform import UniformBAI
from tqdm import tqdm
import pickle

# overall number of active connections
def overall_connections(sim):
    return sum([len(conns) for conns in sim.connections.values()])

# monitors utility per user equipment
def user_utility(sim):
    return {ue.ue_id: utility for ue, utility in sim.utilities.items()}

# monitors each user equipments' distance to their closest base station
def user_closest_distance(sim):
    # position vector of basestations
    bpos = np.array([[bs.x, bs.y] for bs in sim.stations.values()])

    distances = {}    
    for ue_id, ue in sim.users.items():
        upos = np.array([[ue.x, ue.y]])
        dist = np.sqrt(np.sum((bpos - upos)**2, axis=1)).min()
        distances[ue_id] = dist
    return distances

# number of connections per basestation
def station_connections(sim):
    return {bs.bs_id: len(conns) for bs, conns in sim.connections.items()}

def generate_random_positions_inside_circle(radius, num_points, shift):
    # Generate random angles
    random_angles = np.linspace(0, 2*np.pi, num_points)

    # Generate random radius with a slightly reduced range to avoid points clustering at the center
    random_radius = np.sqrt(np.random.uniform(0, 1, num_points)) * radius

    # Convert polar coordinates to Cartesian coordinates
    x = random_radius * np.cos(random_angles) + shift
    y = random_radius * np.sin(random_angles) + shift
    return x, y

def generate_random_positions_inside_circle_min_radius(radius, num_points, s, min_distance):
    # Generate random angles
    random_angles = np.linspace(0, 2*np.pi, num_points)

    # Generate random radius with a slightly reduced range to avoid points clustering at the center
    random_radius = np.sqrt(np.random.uniform(0, 1, num_points)) * radius

    # Convert polar coordinates to Cartesian coordinates
    x = random_radius * np.cos(random_angles) + s
    y = random_radius * np.sin(random_angles) + s
    
    # Calculate initial shift
    shift = np.random.uniform(-radius, radius, 2)

    # Iterate to find shift that satisfies minimum distance requirement
    while np.min(np.sqrt((x - shift[0])**2 + (y - shift[1])**2)) < min_distance:
        shift = np.random.uniform(s-radius,s+ radius, 2)

    x += shift[0]
    y += shift[1]

    return x, y, shift

# Generate simulation environment and parameters
## Configuration. Consider a grid of size 200x200.
config = {
    'seed': 100, 'height': 200, 'width': 200,
    'EP_MAX_TIME': 100,
    "ue": { "velocity": 1.5},
    "metrics": {
        "scalar_metrics": {"overall connections": overall_connections},
        "ue_metrics": {"user utility": user_utility, 'distance station': user_closest_distance},
        "bs_metrics": {"station connections": station_connections}
    }
}

In [ ]:
# Random circle positions
# Initial UEs positions. Spawn the UEs on a circle with the same radius, but different random angles.
SEED = 7
np.random.seed(SEED)
RENDER = True
N_UEs = 10
angles = np.linspace(0, 2 * np.pi - 2 * np.pi/N_UEs, N_UEs) # np.random.uniform(-np.pi, np.pi, size =N_UEs)
K = N_UEs
radiuses = [40,50]
THETAs = []

for radius in radiuses:

    center_x = 100 + radius * np.cos(angles) + 15*np.random.rand(N_UEs)
    center_y = 100 + radius * np.sin(angles) + 15*np.random.rand(N_UEs)
    position_gen = [(lambda x, i=i: (center_x[i], center_y[i])) for i in range(N_UEs)]
    waypoint_gen = None

    # Specify movement class (steady UEs)
    config['movement'] = partial(
        RandomMovement, initial_position_generator=position_gen, waypoint_position_generator=waypoint_gen)

    # Specify UEs
    env = SingleBSMComEnv(render_mode="rgb_array", num_ues=N_UEs, config=config)
    THETA = np.zeros(N_UEs)
    
    for i in range(N_UEs):
        dummy_action = np.zeros(N_UEs)
        obs, _ = env.reset()
        dummy_action[i] = 1
        obs, reward, terminated, truncated, info = env.step(dummy_action)
        scalar_results, ue_results, bs_results = env.monitor.load_results()
        idxs = ue_results['user utility'] > - 1
        ue_results_filtered = ue_results[idxs]
        THETA[i] = scalar_results["mean datarate"].values[0]
        if RENDER:
            plt.imshow(env.render())
            display.display(plt.gcf())
            display.clear_output(wait=True)
    
    THETAs.append(THETA)
    reward_model = CustomRewardModel(THETA)
    instance = MAB(reward_model=reward_model, fairness_model=None)

    SOLVER = cp.ECOS
    [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
    #plt.figure()
    #plt.plot(np.arange(1,K+1),w,label = "$w^\star$")
    #plt.legend()
    print(f"T*_{{theta}} = {sol}")
    
with open('models/scheduling_env_model/circle_models.pkl', 'wb') as handle:
    pickle.dump(THETAs, handle, protocol=pickle.HIGHEST_PROTOCOL)
